In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install --upgrade tables

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.4.4)


In [3]:
import pandas as pd
import numpy as np
from nltk.sentiment import SentimentAnalyzer
import unicodedata
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [0]:
#Path here

#Path here

base_path = '/content/drive/My Drive/stocks_project/'
sp500_path = '/content/drive/My Drive/stocks_project/data/sp/^GSPC.csv'
created_data_path = base_path + 'data/created/data500.h5'
created_data_path_new = base_path + 'data/created/data500_new.h5'
technical500_path = created_data_path + 'technical500.h5'
news_path = '/content/drive/My Drive/stocks_project/data/news/'

In [0]:

import pandas as pd
data = pd.read_hdf(created_data_path);

# Import module

In [6]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

# Import CSV file with all technical data and news

In [0]:
# encoding to avoid UnicodeDecodeError
#data = pd.read_hdf("data.h5", "data")

In [8]:
data.head()

,Open,High,Low,Close,Adj Close,Volume,lrets,MACD,stochastics,ATR,News
1990-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Warm-Weather Sissies?A Proud Beginning to 1990...
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Guest Supply Inc reports earnings for Qtr to S...
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"For Judaism's Remnant, Coup Is Mixed BlessingP..."
1990-01-04,358.760010,358.760010,352.890015,355.670013,355.670013,177000000.0,-0.008650,0.000000,NaN,5.869995,Group W Sports GainsCooney's Common Denominato...
1990-01-05,355.670013,355.670013,351.350006,352.200012,352.200012,158530000.0,-0.009804,-0.036878,NaN,4.320007,President Wins Bipartisan Praise For Solution ...


# Remove data first day

In [9]:
data = data.ix["1990-01-02":]
data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,Open,High,Low,Close,Adj Close,Volume,lrets,MACD,stochastics,ATR,News
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Guest Supply Inc reports earnings for Qtr to S...
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"For Judaism's Remnant, Coup Is Mixed BlessingP..."
1990-01-04,358.760010,358.760010,352.890015,355.670013,355.670013,177000000.0,-0.008650,0.000000,NaN,5.869995,Group W Sports GainsCooney's Common Denominato...
1990-01-05,355.670013,355.670013,351.350006,352.200012,352.200012,158530000.0,-0.009804,-0.036878,NaN,4.320007,President Wins Bipartisan Praise For Solution ...
1990-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CorrectionsFrom Jewish Eden to EmbarrassmentAw...


# Remove holiday 
Feel free to use forward fill to replace Nan value

In [10]:
data.dropna(axis=0, inplace=True)
data.head()

,Open,High,Low,Close,Adj Close,Volume,lrets,MACD,stochastics,ATR,News
1990-01-23,330.380005,332.760010,328.670013,331.609985,331.609985,179300000.0,0.003716,-1.021436,4.863502,4.089997,Counsel Due For Inquiry On DinkinsMadrid Jolte...
1990-01-24,331.609985,331.709991,324.170013,330.260010,330.260010,207830000.0,-0.004079,-1.103580,0.000000,7.539978,Books of The Times; A Satire on the Brezhnev...
1990-01-25,330.260010,332.329987,325.329987,326.079987,326.079987,172270000.0,-0.012738,-1.323376,0.000000,7.000000,Miami Officer Is Sentenced to 7 Years for 2 Ki...
1990-01-26,326.089996,328.579987,321.440002,325.799988,325.799988,198190000.0,-0.000859,-1.419756,0.000000,7.139985,Threat of Oil Shortage by '93 SeenCarol Wincen...
1990-01-29,325.799988,327.309998,321.790009,325.200012,325.200012,150770000.0,-0.001843,-1.447355,0.000000,5.519989,1855 Louisiana Law On Abortion Is BarredManila...


# Removing open high low close

In [11]:
#data.drop(['close', 'high', 'low', 'open','adj_factor', 'adj_high', 'adj_low', 'adj_open', 'adj_volume', 'ex_dividend', 'split_ratio'], axis=1, inplace=True)


data.drop(['Open', 'High', 'Low', 'Close'], axis=1, inplace=True)
data.head()

,Adj Close,Volume,lrets,MACD,stochastics,ATR,News
1990-01-23,331.609985,179300000.0,0.003716,-1.021436,4.863502,4.089997,Counsel Due For Inquiry On DinkinsMadrid Jolte...
1990-01-24,330.260010,207830000.0,-0.004079,-1.103580,0.000000,7.539978,Books of The Times; A Satire on the Brezhnev...
1990-01-25,326.079987,172270000.0,-0.012738,-1.323376,0.000000,7.000000,Miami Officer Is Sentenced to 7 Years for 2 Ki...
1990-01-26,325.799988,198190000.0,-0.000859,-1.419756,0.000000,7.139985,Threat of Oil Shortage by '93 SeenCarol Wincen...
1990-01-29,325.200012,150770000.0,-0.001843,-1.447355,0.000000,5.519989,1855 Louisiana Law On Abortion Is BarredManila...


# Using NLTK sentiment analyzer to generate a polarity score

Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

https://www.nltk.org/_modules/nltk/sentiment/sentiment_analyzer.html

In [0]:
def sentimentanalysis(df):
    sid = SentimentIntensityAnalyzer()
    for date in df.index:
        try:
            sentence = unicodedata.normalize('NFKD', df.loc[date, 'News'])
            ss = sid.polarity_scores(sentence)
            neg_val = ss['neg']
            neu_val = ss['neu']
            pos_val = ss['pos']
            df.at[date, 'neg'] = neg_val
            df.at[date, 'neu'] = neu_val
            df.at[date, 'pos'] = pos_val
            
        except TypeError:
            print (df.loc[date, 'News'])
            print (date)
    return df

In [0]:
new_data = sentimentanalysis(data)

In [0]:
new_data.drop(['News'], axis=1, inplace=True)

In [0]:
#new_data.to_hdf('data2', 'new_data')
new_data.to_hdf(created_data_path_new, 'new_data')

# Check if there is missing data

In [16]:
new_data.isnull().any()

Adj Close      False
Volume         False
lrets          False
MACD           False
stochastics    False
ATR            False
neg            False
neu            False
pos            False
dtype: bool